# PAN - Atribuição Autoral - 2018 - Modelo Char wb



In [1]:
%matplotlib inline

In [2]:
#python basic libs
import os;
from os.path import join as pathjoin;
import zipfile;

import re;
import random;
import json;
from collections import defaultdict, Counter;
from pprint import pprint
from time import time

In [3]:
#data analysis libs
import numpy as np;
import pandas as pd;
from pandas.plotting import scatter_matrix;
import matplotlib.pyplot as plt;


import seaborn as sns;

In [4]:
#machine learning libs
#feature extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import feature_selection;

#preprocessing and transformation
from sklearn import preprocessing;
from sklearn.preprocessing import MaxAbsScaler;
from sklearn.decomposition import PCA;

from sklearn.base import BaseEstimator, ClassifierMixin

#classifiers
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression


#
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

#model valuation
from sklearn.model_selection import train_test_split;
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score;

NLP Libs

In [5]:
# import spacy;
# import nltk;
# import gensim
# import logging
# from gensim.models import KeyedVectors
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.CRITICAL)

In [6]:
from IPython.display import Markdown, display, HTML

libraries configurations

In [7]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.simplefilter(action='ignore', category=UndefinedMetricWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [8]:
np.set_printoptions(precision=4)
pd.options.display.float_format = '{:,.4f}'.format
sns.set(color_codes=True);

In [9]:
import platform;
import sklearn;
import scipy;

Markdown(
    "|%-15s|%-65s|\n"%("PACK","VERSION") +
    "|%-15s|%-65s|\n"%('-'*15,'-'*65) +
    ('\n'.join(
    "|%-15s|%-45s| " % (pack, version.strip())
    for pack, version in
    zip(['SO','NumPy','SciPy','Scikit-Learn','Seaborn'],
        [platform.platform(), np.__version__, scipy.__version__,
         sklearn.__version__, sns.__version__])

)))

|PACK           |VERSION                                                          |
|---------------|-----------------------------------------------------------------|
|SO             |Darwin-18.2.0-x86_64-i386-64bit              | 
|NumPy          |1.15.4                                       | 
|SciPy          |1.1.0                                        | 
|Scikit-Learn   |0.20.1                                       | 
|Seaborn        |0.9.0                                        | 

In [10]:
import seaborn as sns;
sns.set(color_codes=True);

In [11]:
import platform; print(platform.platform())
print("NumPy", np.__version__)
import scipy; print("SciPy", scipy.__version__)
import sklearn; print("Scikit-Learn", sklearn.__version__)
print("seaborn", sns.__version__)

Darwin-18.2.0-x86_64-i386-64bit
NumPy 1.15.4
SciPy 1.1.0
Scikit-Learn 0.20.1
seaborn 0.9.0


In [12]:
np.set_printoptions(precision=4)
pd.options.display.float_format = '{:,.4f}'.format

In [13]:
from skleanExtensions import DenseTransformer

### paths configuration

In [14]:
baseDir = '../';

corpusTraining    = 'pan18-cross-domain-authorship-attribution-training-dataset-2017-12-02';
corpusEvaluation = 'pan18-cross-domain-authorship-attribution-test-dataset2-2018-04-20';
corpusEach1 = 'Lyrics_AA_PT';

currentCorpus = corpusTraining;

inputDir= pathjoin(baseDir,currentCorpus);
outputDir= pathjoin(baseDir,'out');
if not os.path.exists(outputDir):
    os.mkdir(outputDir);

## loading the dataset

In [15]:
import pan

In [16]:
problems = pan.readCollectionsOfProblemsFromZip(inputDir + '.zip');

In [17]:
problems[0]['problem']

'problem00001'

In [18]:
pd.DataFrame(problems)[['problem','language','authorCount','candidates','unknown']]

,problem,language,authorCount,candidates,unknown
0,problem00001,en,20,"[[graceful ones.\n\n""One more,"" Marvelous said...","[[a word it said, not wanting to see them anym..."
1,problem00002,en,5,"[[a mission.""\n\nJensen just raises an eyebrow...","[[don't mean you, technically, but it's just t..."
2,problem00003,fr,20,[[qui l'avait tué mais tout était de la faute ...,[[scène qui se déroulait entre leurs bras. Le ...
3,problem00004,fr,5,[[. Le canapé est vide et lorsqu'il passe deva...,"[[lui. Le silence m'insupporte et rapidement, ..."
4,problem00005,it,20,"[[Eppure lui la mappa l’aveva stampata, dannaz...",[[nella sua prima casa da solo.\nAveva usato a...
5,problem00006,it,5,[[Yato ha trovato una lettera sul suo comodino...,"[[Bah, dà qua, che peggio di così non può anda..."
6,problem00007,pl,20,[[zmienił zdanie. Niech się stworzonko pobawi....,"[[Przynajmniej tak sądził chłopak, dopóki nie ..."
7,problem00008,pl,5,"[[Słowem, które Sherlock najczęściej słyszał w...",[[zamknął oczy. Serce waliło mu jak oszalałe; ...
8,problem00009,sp,20,[[pero no lo ama como ama a Guignol –explicó e...,"[[de tanto contacto entre ellos, fue que Albus..."
9,problem00010,sp,5,"[[incapaz de señalar un momento exacto, un pun...","[[editores son magos, así que tiene el peor hu..."


### examinando o parametro min_df isoladamente

In [19]:
def runML(problem):
    print ("\nProblem: %s,  language: %s, " %(problem['problem'],problem['language']), end=' ')
    
    train_docs, train_labels, _  = zip(*problem['candidates'])
    test_docs, test_labels, test_filename   = zip(*problem['unknown'])
    
    pipeline = Pipeline([
        ('vect',   TfidfVectorizer(analyzer='word',
                                   max_df=1.0,
                                   norm='l1',
                                   ngram_range=(1,3),
                                   sublinear_tf=True,
                                   smooth_idf=True,
                                   lowercase =False)),
        ('dense',  DenseTransformer()),
        ('scaler', MaxAbsScaler()),
        ('transf', PCA(0.999)),
        ('clf', LogisticRegression(random_state=0,multi_class='multinomial', solver='newton-cg')),
    ])
    
    # uncommenting more parameters will give better exploring power but will
    # increase processing time in a combinatorial way
    parameters = {
        'vect__min_df':(0.01,0.05,0.1)
    }
    
    grid_search = GridSearchCV(pipeline,
                               parameters,
                               cv=3,
                               iid=False,
                               verbose=False,
                               scoring='f1_macro'
                              )
    
    t0 = time()
    grid_search.fit(train_docs, train_labels)
    print("GridSearch: %0.3fs" % (time() - t0), end=' ')
    print("Best score: %0.3f" % grid_search.best_score_, end=' ')
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    train_pred=grid_search.predict(train_docs);
    test_pred=grid_search.predict(test_docs);
    
    
    f1 = f1_score(test_labels,test_pred, average='macro')
    precision = precision_score(test_labels,test_pred, average='macro')
    recall =  recall_score(test_labels,test_pred, average='macro')
    accuracy =   accuracy_score(test_labels,test_pred)
    
    return {
                'problem'  :       problem['problem'],
                "language"      :       problem['language'],
                'AuthorCount'   :       len(set(train_labels)),
                "train_doc_size":       len(train_docs),
                "train_caract_per_doc": sum([len(l) for l in train_docs])/len(train_docs),
                "test_doc_size" :       len(test_docs),
                "test_caract_per_doc":  sum([len(l) for l in test_docs])/len(test_docs),
                
                'f1'       : round(f1,3),
                'precision': round(precision,3),
                'recall'   : round(recall,3),
                'accuracy' : round(accuracy,3),
                
        }, grid_search.cv_results_, best_parameters;

In [20]:
result = [];
cv_result = [];
best_parameters = [];
for problem in problems:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore");
        r, c, b = runML(problem);
        result.append(r);
        cv_result.append(c);
        b['problem'] = problem['problem'];
        best_parameters.append(b);


Problem: problem00001,  language: en,  GridSearch: 5.830s Best score: 0.462 Best parameters set:
	vect__min_df: 0.01

Problem: problem00002,  language: en,  GridSearch: 1.046s Best score: 0.664 Best parameters set:
	vect__min_df: 0.1

Problem: problem00003,  language: fr,  GridSearch: 4.582s Best score: 0.563 Best parameters set:
	vect__min_df: 0.05

Problem: problem00004,  language: fr,  GridSearch: 0.990s Best score: 0.762 Best parameters set:
	vect__min_df: 0.01

Problem: problem00005,  language: it,  GridSearch: 5.799s Best score: 0.365 Best parameters set:
	vect__min_df: 0.01

Problem: problem00006,  language: it,  GridSearch: 1.322s Best score: 0.644 Best parameters set:
	vect__min_df: 0.01

Problem: problem00007,  language: pl,  GridSearch: 5.179s Best score: 0.447 Best parameters set:
	vect__min_df: 0.05

Problem: problem00008,  language: pl,  GridSearch: 1.145s Best score: 0.500 Best parameters set:
	vect__min_df: 0.01

Problem: problem00009,  language: sp,  GridSearch: 6.135

In [22]:
for c,p in zip(cv_result, problems):
    c['problem'] = p['problem']
df =  pd.concat([
    pd.DataFrame(c) for c in cv_result
])
df = df[df.rank_test_score == 1]

df = df[['problem', 'mean_test_score','std_test_score','param_vect__min_df']]
df.sort_values(['problem','mean_test_score','param_vect__min_df'])

,problem,mean_test_score,std_test_score,param_vect__min_df
0,problem00001,0.4616,0.0532,0.0100
2,problem00002,0.6644,0.0974,0.1000
1,problem00003,0.5628,0.0384,0.0500
0,problem00004,0.7622,0.2049,0.0100
1,problem00004,0.7622,0.2049,0.0500
0,problem00005,0.3652,0.0630,0.0100
0,problem00006,0.6444,0.1257,0.0100
1,problem00006,0.6444,0.1257,0.0500
1,problem00007,0.4470,0.0804,0.0500
0,problem00008,0.5000,0.0000,0.0100


### analisando os demais parametros

In [28]:
def runML(problem):
    print ("\nProblem: %s,  language: %s, " %(problem['problem'],problem['language']), end=' ')
    
    train_docs, train_labels, _ = zip(*problem['candidates'])
    test_docs, test_labels, test_filename = zip(*problem['unknown'])
    
    pipeline = Pipeline([
        ('vect',   TfidfVectorizer(analyzer='word',
                                   min_df=0.01,
                                   max_df=1.0,
                                   norm='l1')),
        ('scaler', MaxAbsScaler()),
        ('dense',  DenseTransformer()),
        ('transf', PCA(0.999)),
        ('clf', LogisticRegression(random_state=0,multi_class='multinomial', solver='newton-cg')),
    ])
    
    
    # uncommenting more parameters will give better exploring power but will
    # increase processing time in a combinatorial way
    parameters = {
        'vect__ngram_range':((1,1),(1,2),(1,3)),
        'vect__use_idf'     :(True, False),
        'vect__sublinear_tf':(True, False),
        'vect__smooth_idf'  :(True, False),
        'vect__norm'        :('l1','l2'),
        'vect__lowercase'   :(True,False)
    }
    
    grid_search = GridSearchCV(pipeline,
                               parameters,
                               cv=4,
                               iid=False,
                               n_jobs=-1,
                               verbose=False,
                               scoring='f1_macro')
    
    t0 = time()
    grid_search.fit(train_docs, train_labels)
    print("Gridsearh %0.3fs" % (time() - t0), end=' ')

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    print("'"+problem['language']+"':{")
    for param_name in sorted(parameters.keys()):
        print("\t'%s': %r," % (param_name, best_parameters[param_name]))
    print("},")
    
    train_pred=grid_search.predict(train_docs);
    test_pred=grid_search.predict(test_docs);
    
        
    train_pred=grid_search.predict(train_docs);
    test_pred=grid_search.predict(test_docs);
    
    
    f1        = f1_score       (test_labels,test_pred, average='macro');
    precision = precision_score(test_labels,test_pred, average='macro');
    recall    = recall_score   (test_labels,test_pred, average='macro');
    accuracy  = accuracy_score (test_labels,test_pred);
    
    def mean(a):
        return sum(a)/len(a);
    
    
    return {
                'problem'  :       problem['problem'],
                "language"      :       problem['language'],
                'authorCount'   :       len(set(train_labels)),
        
                "train_doc_size":       len(train_docs),
                "train_doc_per_author": mean(Counter(train_labels).values()) ,
                "train_caract_per_doc": sum([len(l) for l in train_docs])/len(train_docs),        
        
                "test_doc_size" :       len(test_docs),
                "test_doc_per_author": mean(Counter(test_labels).values()) ,
                "test_caract_per_doc":  sum([len(l) for l in test_docs])/len(test_docs),
                
                'f1'       : round(f1,3),
                'precision': round(precision,3),
                'recall'   : round(recall,3),
                'accuracy' : round(accuracy,3),
                
        }, grid_search.cv_results_,best_parameters, grid_search.best_estimator_;

In [29]:
result = [];
cv_result = [];
best_parameters = [];
estimators = [];
for problem in problems:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore");
        r, c, b, e = runML(problem);
        result.append(r);
        cv_result.append(c);
        estimators.append(e);
        b['problem'] = problem['problem'];
        best_parameters.append(b);


Problem: problem00001,  language: en,  Gridsearh 104.727s Best score: 0.542
Best parameters set:
'en':{
	'vect__lowercase': True,
	'vect__ngram_range': (1, 2),
	'vect__norm': 'l1',
	'vect__smooth_idf': True,
	'vect__sublinear_tf': False,
	'vect__use_idf': True,
},

Problem: problem00002,  language: en,  Gridsearh 16.187s Best score: 0.800
Best parameters set:
'en':{
	'vect__lowercase': True,
	'vect__ngram_range': (1, 1),
	'vect__norm': 'l1',
	'vect__smooth_idf': True,
	'vect__sublinear_tf': True,
	'vect__use_idf': True,
},

Problem: problem00003,  language: fr,  Gridsearh 105.641s Best score: 0.631
Best parameters set:
'fr':{
	'vect__lowercase': True,
	'vect__ngram_range': (1, 3),
	'vect__norm': 'l2',
	'vect__smooth_idf': True,
	'vect__sublinear_tf': True,
	'vect__use_idf': True,
},

Problem: problem00004,  language: fr,  Gridsearh 14.667s Best score: 0.808
Best parameters set:
'fr':{
	'vect__lowercase': True,
	'vect__ngram_range': (1, 2),
	'vect__norm': 'l1',
	'vect__smooth_idf': Tru

In [32]:
df=pd.DataFrame(result)[['problem',
                     "language",
                     'authorCount',
                     "train_doc_size",
                     "test_doc_size", 
                     'f1','precision','recall' ,'accuracy']]
df

,problem,language,authorCount,train_doc_size,test_doc_size,f1,precision,recall,accuracy
0,problem00001,en,20,80,105,0.3790,0.3970,0.4970,0.5140
1,problem00002,en,5,20,21,0.4970,0.4670,0.5670,0.7620
2,problem00003,fr,20,80,49,0.2790,0.3260,0.3230,0.3470
3,problem00004,fr,5,20,21,0.3590,0.4190,0.4270,0.4290
4,problem00005,it,20,80,80,0.3850,0.4010,0.5640,0.4120
5,problem00006,it,5,20,46,0.4330,0.5120,0.7170,0.5430
6,problem00007,pl,20,80,103,0.3130,0.3540,0.3290,0.3790
7,problem00008,pl,5,20,15,0.6110,0.6780,0.5780,0.7330
8,problem00009,sp,20,80,117,0.4670,0.4950,0.5900,0.5470
9,problem00010,sp,5,20,64,0.5890,0.6250,0.6080,0.6560


In [33]:
df[['f1']].mean()

f1   0.4312
dtype: float64

In [34]:
languages={
    'en':'inglesa',
    'sp':'espanhola',
    'it':'italiana',
    'pl':'polonesa',
    'fr':'francesa'
}

In [35]:
for c,p in zip(cv_result, problems):
    c['problem'] = p['problem']
    c['authorCount'] = p ['authorCount']
    c['language'] = p ['language']
dfCV =  pd.concat([
    pd.DataFrame(c) for c in cv_result
])

params = {
    'param_vect__ngram_range':'ngram_range',
    'param_vect__sublinear_tf':'sublinear_tf',
    'param_vect__use_idf':'use_idf',
    'param_vect__norm':'norm',
    'param_vect__lowercase':'lowercase'
}
dfCV = dfCV[['problem','language', 'authorCount','rank_test_score', 'mean_test_score','std_test_score'] + list(params.keys())].rename(columns=params)
dfCV.problem = dfCV.problem.apply(lambda x: re.sub(r'\D','',x))
dfCV.sort_values(['problem','mean_test_score'],ascending=[True,False], inplace=True)
dfCV[dfCV.rank_test_score == 1]

,problem,language,authorCount,rank_test_score,mean_test_score,std_test_score,ngram_range,sublinear_tf,use_idf,norm,lowercase
18,00001,en,20,1,0.5417,0.1066,"(1, 2)",False,True,l1,True
22,00001,en,20,1,0.5417,0.1066,"(1, 2)",False,True,l1,True
0,00002,en,5,1,0.8000,0.1155,"(1, 1)",True,True,l1,True
4,00002,en,5,1,0.8000,0.1155,"(1, 1)",True,True,l1,True
40,00003,fr,20,1,0.6312,0.1128,"(1, 3)",True,True,l2,True
44,00003,fr,20,1,0.6312,0.1128,"(1, 3)",True,True,l2,True
17,00004,fr,5,1,0.8083,0.2087,"(1, 2)",True,False,l1,True
21,00004,fr,5,1,0.8083,0.2087,"(1, 2)",True,False,l1,True
24,00004,fr,5,1,0.8083,0.2087,"(1, 2)",True,True,l2,True
25,00004,fr,5,1,0.8083,0.2087,"(1, 2)",True,False,l2,True


## Saving the model

In [ ]:
dfCV.to_csv('PANAA2018_word.csv', index=False,compression='zip')

In [ ]:
dfCV = pd.read_csv('PANAA2018_word.csv', na_values='',compression='zip')

In [ ]:
import pickle;
with open("PANAA2018_word.pkl","wb") as f:
    pickle.dump(estimators,f)

## understanding the model with reports

Podemos ver que para um mesmo problema mais de uma configuração é possível

In [ ]:
with pd.option_context('display.precision', 4):
    html = (dfCV.pivot_table(
                index=['problem','language'],
                columns=['ngram_range','lowercase','use_idf','sublinear_tf','norm'],
                values='mean_test_score',aggfunc='mean'
                
            ).T.style.background_gradient(axis=0, cmap=plt.cm.Blues))
html

In [ ]:
pd.options.display.precision = 3  
print(u"\\begin{table}[h]\n\\centering\n\\caption{Medida F1 para os parâmetros }")

print(re.sub(r'[ ]{2,}',' ',dfCV.pivot_table(
                index=['problem','language'],
                columns=['ngram_range','sublinear_tf','use_idf','norm'],
                values='mean_test_score',aggfunc='mean'
                
            ).to_latex()))
print ("\label{tab:modelocaracter}")
print(r"\end{table}")

In [ ]:
dfCV.ngram_range[0]

In [ ]:
d = dfCV.copy()
d = d.rename(columns={'language':u'Língua', 'sublinear_tf':'TF Sublinear'})
d = d [ d.norm.isna() == False]
d.problem = d.apply(lambda x: x[u'Língua'] +" "+ str(x[u'problem']), axis=1)
d.ngram_range = d.ngram_range.apply(lambda x: int(re.sub(r'(.*,)|\D',r'',x)))
#d.ngram_range = d.apply(lambda x: str(x[u'ngram_range'][0]) +" "+ str(x[u'ngram_range'][1]), axis=1)

d.std_test_score =d.std_test_score / d.std_test_score.quantile(0.95) *500;
d.std_test_score +=1;
d.std_test_score = d.std_test_score.astype(np.int64)
g = sns.FacetGrid(d, col='Língua', hue='TF Sublinear',  row="norm", height=3,palette="Set1")
g.map(plt.scatter, "ngram_range", "mean_test_score",alpha=0.5, s=d.std_test_score.values/2).add_legend();
#sns.pairplot(d, hue="TF Sublinear", vars=["autorNumber", "mean_test_score"])


In [ ]:
g = sns.FacetGrid(d, row='authorCount', hue='TF Sublinear',  col=u"Língua", height=3,palette="Set1")
g.map(plt.scatter, "ngram_range", "mean_test_score", alpha=0.5, s=d.std_test_score.values/2).add_legend();

In [ ]:
sns.distplot(dfCV.std_test_score, bins=50);

In [ ]:
import statsmodels.api as sm

In [ ]:
d = dfCV[['mean_test_score','problem', 'language','sublinear_tf','norm','ngram_range','use_idf','lowercase']].copy();

d.lowercase=d.lowercase.apply(lambda x: 1 if x else 0)
d.sublinear_tf=d.sublinear_tf.apply(lambda x: 1 if x else 0)
d.use_idf     =d.use_idf.apply(lambda x: 1 if x else 0)
d.norm        =d.norm.apply(lambda x: 1 if x == 'l1' else 0)

d.norm.fillna(value='None', inplace=True);

d.ngram_range = d.ngram_range.apply(lambda x: int(re.sub(r'(.*,)|\D',r'',x))).astype(np.uint8)
d.drop(columns=['problem'], inplace=True)
d['intercept'] = 1;

#d.mean_test_score = np.log(d.mean_test_score/(1-d.mean_test_score))

d=pd.get_dummies(d, columns=['language'])

In [ ]:
d.describe()

In [ ]:
mod = sm.OLS( d.iloc[:,0], d.iloc[:,1:])
res = mod.fit()
res.summary()

In [ ]:
sns.distplot(res.predict()-d.iloc[:,0].values, bins=25);

In [ ]:
sns.jointplot(x='F1',y='F1-estimated',data=pd.DataFrame({'F1':d.iloc[:,0].values, 'F1-estimated':res.predict()}));